# OpaquePrompts

[OpaquePrompts](https://opaqueprompts.readthedocs.io/en/latest/) 是一项服务，它能让应用程序在不损害用户隐私的前提下，利用语言模型的能力。OpaquePrompts 的设计初衷是为了组合性和轻松集成到现有应用程序和服务中，可以通过简单的 Python 库以及 LangChain 来使用。可能更重要的是，OpaquePrompts 利用了[机密计算](https://en.wikipedia.org/wiki/Confidential_computing)的能力，确保即使是 OpaquePrompts 服务本身也无法访问它所保护的数据。
 

本 Notebook 将介绍如何使用 LangChain 来与 `OpaquePrompts` 进行交互。

In [ ]:
# install the opaqueprompts and langchain packages
%pip install --upgrade --quiet  opaqueprompts langchain

访问 OpaquePrompts API 需要一个 API 密钥，您可以通过在 [OpaquePrompts 网站](https://opaqueprompts.opaque.co/) 上创建账户来获取。一旦您拥有账户，就可以在 [API 密钥页面](https:opaqueprompts.opaque.co/api-keys) 上找到您的 API 密钥。

In [ ]:
import os

# Set API keys

os.environ["OPAQUEPROMPTS_API_KEY"] = "<OPAQUEPROMPTS_API_KEY>"
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

# 使用 OpaquePrompts LLM 封装器

将 OpaquePrompts 应用到您的应用程序可能非常简单，只需将您的 LLM 包装在 OpaquePrompts 类中，用 `llm=OpaquePrompts(base_llm=OpenAI())` 替换 `llm=OpenAI()` 即可。

In [ ]:
from langchain.chains import LLMChain
from langchain.globals import set_debug, set_verbose
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import OpaquePrompts
from langchain_core.callbacks import StdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

set_debug(True)
set_verbose(True)

prompt_template = """
As an AI assistant, you will answer questions according to given context.

Sensitive personal information in the question is masked for privacy.
For instance, if the original text says "Giana is good," it will be changed
to "PERSON_998 is good." 

Here's how to handle these changes:
* Consider these masked phrases just as placeholders, but still refer to
them in a relevant way when answering.
* It's possible that different masked terms might mean the same thing.
Stick with the given term and don't modify it.
* All masked terms follow the "TYPE_ID" pattern.
* Please don't invent new masked terms. For instance, if you see "PERSON_998,"
don't come up with "PERSON_997" or "PERSON_999" unless they're already in the question.

Conversation History: ```{history}```
Context : ```During our recent meeting on February 23, 2023, at 10:30 AM,
John Doe provided me with his personal details. His email is johndoe@example.com
and his contact number is 650-456-7890. He lives in New York City, USA, and
belongs to the American nationality with Christian beliefs and a leaning towards
the Democratic party. He mentioned that he recently made a transaction using his
credit card 4111 1111 1111 1111 and transferred bitcoins to the wallet address
1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa. While discussing his European travels, he noted
down his IBAN as GB29 NWBK 6016 1331 9268 19. Additionally, he provided his website
as https://johndoeportfolio.com. John also discussed some of his US-specific details.
He said his bank account number is 1234567890123456 and his drivers license is Y12345678.
His ITIN is 987-65-4321, and he recently renewed his passport, the number for which is
123456789. He emphasized not to share his SSN, which is 123-45-6789. Furthermore, he
mentioned that he accesses his work files remotely through the IP 192.168.1.1 and has
a medical license number MED-123456. ```
Question: ```{question}```

"""

chain = LLMChain(
    prompt=PromptTemplate.from_template(prompt_template),
    llm=OpaquePrompts(base_llm=OpenAI()),
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True,
)


print(
    chain.run(
        {
            "question": """Write a message to remind John to do password reset for his website to stay secure."""
        },
        callbacks=[StdOutCallbackHandler()],
    )
)

从输出中，您可以看到来自用户输入的以下上下文包含敏感数据。

```
# 用户输入中的上下文

在我们最近于 2023 年 2 月 23 日上午 10:30 举行的会议上，John Doe 向我提供了他的个人详细信息。他的电子邮件是 johndoe@example.com，联系电话是 650-456-7890。他住在美利坚合众国纽约市，信仰基督教，倾向于民主党。他提到最近使用信用卡 4111 1111 1111 1111 进行了一笔交易，并将比特币转入钱包地址 1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa。在讨论他的欧洲旅行时，他记下了他的 IBAN 号码 GB29 NWBK 6016 1331 9268 19。此外，他还提供了他的网站 https://johndoeportfolio.com。John 还讨论了他的一些美国特定详情。他说他的银行账号是 1234567890123456，他的驾照是 Y12345678。他的 ITIN 是 987-65-4321，他最近更新了护照，护照号码是 123456789。他强调不要分享他的 SSN，即 669-45-6789。此外，他还提到他通过 IP 地址 192.168.1.1 远程访问他的工作文件，并且他的医疗执照号码是 MED-123456。
```

OpaquePrompts 将自动检测敏感数据并将其替换为占位符。

```
# OpaquePrompts 处理后的上下文

在我们最近于 DATE_TIME_3，于 DATE_TIME_2 举行的会议上，PERSON_3 向我提供了他的个人详细信息。他的电子邮件是 EMAIL_ADDRESS_1，联系电话是 PHONE_NUMBER_1。他住在 LOCATION_3，LOCATION_2，属于 NRP_3 国籍，拥有 NRP_2 信仰，倾向于民主党。他提到最近使用信用卡 CREDIT_CARD_1 进行了一笔交易，并将比特币转入钱包地址 CRYPTO_1。在讨论他的 NRP_1 旅行时，他记下了他的 IBAN 号码 IBAN_CODE_1。此外，他还提供了他的网站 URL_1。PERSON_2 还讨论了他的一些 LOCATION_1 特定详情。他说他的银行账号是 US_BANK_NUMBER_1，他的驾照是 US_DRIVER_LICENSE_2。他的 ITIN 是 US_ITIN_1，他最近更新了护照，护照号码是 DATE_TIME_1。他强调不要分享他的 SSN，即 US_SSN_1。此外，他还提到他通过 IP 地址 IP_ADDRESS_1 远程访问他的工作文件，并且他的医疗执照号码是 MED-US_DRIVER_LICENSE_1。
```

占位符已在 LLM 的响应中使用。

```
# LLM 返回的响应

嘿 PERSON_1，只是想提醒你通过你的电子邮件 EMAIL_ADDRESS_1 为你的网站 URL_1 重置密码。在线保持安全很重要，所以别忘了做！
```

响应通过用原始敏感数据替换占位符进行了脱敏处理。

```
# 来自 OpaquePrompts 的脱敏 LLM 响应

嘿 John，只是想提醒你通过你的电子邮件 johndoe@example.com 为你的网站 https://johndoeportfolio.com 重置密码。在线保持安全很重要，所以别忘了做！
```

# 在 LangChain 表达式中使用 OpaquePrompts

如果直接替换无法满足您的灵活性需求，您也可以在 LangChain 表达式中使用这些函数。

In [ ]:
import langchain_community.utilities.opaqueprompts as op
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = (PromptTemplate.from_template(prompt_template),)
llm = OpenAI()
pg_chain = (
    op.sanitize
    | RunnablePassthrough.assign(
        response=(lambda x: x["sanitized_input"]) | prompt | llm | StrOutputParser(),
    )
    | (lambda x: op.desanitize(x["response"], x["secure_context"]))
)

pg_chain.invoke(
    {
        "question": "Write a text message to remind John to do password reset for his website through his email to stay secure.",
        "history": "",
    }
)